In [10]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.io import FortranFile
from cobaya.model import get_model
import yaml
import os
from getdist import plots, MCSamples
from cobaya.conventions import Const, packages_path_input
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

In [2]:
path_GLM_data = '/home/afarina/ParametricModels/LCDM/GLM_data/'

In [3]:

getdist_settings = {
    'smooth_scale_1D': 0.3,
    'smooth_scale_2D': 0.4,
    'boundary_correction_order': 1,
    'mult_bias_correction_order': 1,
    }


# BAO

In [4]:
yaml_path = "/home/afarina/ParametricModels/LCDM/chains/01p1_LCDM_BAO_DESIDR2.input.yaml"

# define the path to the YAML parameters file:
yaml_params = yaml_path #os.path.join(chains_dir, name + '.input.yaml')
# load the parameters from the YAML file:
with open(yaml_params, 'r') as f:
    params_dict = yaml.safe_load(f)
# remove the 'path' key from the 'camb' dictionary if it exists:
if 'path' in params_dict.get('theory', {}).get('camb', {}):
    del params_dict['theory']['camb']['path']
if 'recombination_model' in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['recombination_model']
if 'max_zrei'in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['max_zrei']
    print('Removed "max_zrei" from the "camb" extra_args in the YAML parameters file.')
# build the model:
model = get_model(params_dict)

Removed "max_zrei" from the "camb" extra_args in the YAML parameters file.
[model] *WARNING* Ignored blocks/options: ['sampler', 'output']
[camb] `camb` module loaded successfully from /home/afarina/anaconda3/envs/nbase/lib/python3.12/site-packages/camb
[bao.desi_dr2.desi_bao_all] Initialized.


In [5]:
chains_dir = os.path.join('../chains')

# import chain
from cobaya import load_samples

name = '01p1_LCDM_BAO_DESIDR2'
chain = load_samples(os.path.join(chains_dir, name), skip=0.3, to_getdist=True)
chain.label = name
chain.root = os.path.join(chains_dir, name)
chain.updateSettings(getdist_settings)
chain.updateBaseStatistics()
chain.getConvergeTests()
print(chain.label+': R =', round(chain.GelmanRubin, 4))


BestFit = chain.getBestFit()
BestFitDict = BestFit.getParamDict()

params = model.parameterization.sampled_params()
for p in params.keys():
    params[p] = BestFitDict[p]  # Set the best fit values for all parameters

[samplecollection] Loaded 4340 sample points from '../chains/01p1_LCDM_BAO_DESIDR2.1.txt'
[samplecollection] Loaded 3987 sample points from '../chains/01p1_LCDM_BAO_DESIDR2.2.txt'
[samplecollection] Loaded 4246 sample points from '../chains/01p1_LCDM_BAO_DESIDR2.3.txt'
[samplecollection] Loaded 4264 sample points from '../chains/01p1_LCDM_BAO_DESIDR2.4.txt'
01p1_LCDM_BAO_DESIDR2: R = 0.0081


## Maximum Likelihood estimator

In [6]:
BAO_pars = model.parameterization.sampled_params().keys()

def loglike_BAO(param_vec):
    par_dict = {}
    for i,key in enumerate(BAO_pars):
        par_dict[key] = param_vec[i]
    
    return model.loglike(par_dict, return_derived=False)


In [11]:
differential_evolution?

Signature:
differential_evolution(
    func,
    bounds,
    args=(),
    strategy='best1bin',
    maxiter=1000,
    popsize=15,
    tol=0.01,
    mutation=(0.5, 1),
    recombination=0.7,
    seed=None,
    callback=None,
    disp=False,
    polish=True,
    init='latinhypercube',
    atol=0,
    updating='immediate',
    workers=1,
    constraints=(),
    x0=None,
    *,
    integrality=None,
    vectorized=False,
)
Docstring:
Finds the global minimum of a multivariate function.

The differential evolution method [1]_ is stochastic in nature. It does
not use gradient methods to find the minimum, and can search large areas
of candidate space, but often requires larger numbers of function
evaluations than conventional gradient-based techniques.

The algorithm is due to Storn and Price [2]_.

Parameters
----------
func : callable
    The objective function to be minimized. Must be in the form
    ``f(x, *args)``, where ``x`` is the argument in the form of a 1-D array
    and ``args`` is

In [13]:
initial_guess = list(params.values())
bounds = [(-5, 5)] * len(initial_guess)
# result = minimize(loglike_BAO, initial_guess)

result = differential_evolution(loglike_BAO, bounds, polish=True, disp=True)

differential_evolution step 1: f(x)= -inf
differential_evolution step 2: f(x)= -inf
differential_evolution step 3: f(x)= -inf
differential_evolution step 4: f(x)= -inf
differential_evolution step 5: f(x)= -inf
differential_evolution step 6: f(x)= -inf
differential_evolution step 7: f(x)= -inf
differential_evolution step 8: f(x)= -inf
differential_evolution step 9: f(x)= -inf
differential_evolution step 10: f(x)= -inf
differential_evolution step 11: f(x)= -inf
differential_evolution step 12: f(x)= -inf
differential_evolution step 13: f(x)= -inf
differential_evolution step 14: f(x)= -inf
differential_evolution step 15: f(x)= -inf
differential_evolution step 16: f(x)= -inf
differential_evolution step 17: f(x)= -inf
differential_evolution step 18: f(x)= -inf
differential_evolution step 19: f(x)= -inf
differential_evolution step 20: f(x)= -inf
differential_evolution step 21: f(x)= -inf
differential_evolution step 22: f(x)= -inf
differential_evolution step 23: f(x)= -inf
differential_evoluti

: 

In [ ]:
cov_data_BAO = np.load(path_GLM_data + "cov_data_BAO.npy")

In [ ]:
Q_ML = - 2*result.fun - np.shape(cov_data_BAO)[0] * np.log(2*np.pi) - np.log(np.linalg.det(cov_data_BAO))
Q_ML

23.408073320874223

In [ ]:
result.x

array([9.7325616e+01, 3.2143883e+00, 9.8924241e-01, 5.8514733e-02,
       2.2246340e-01, 5.3702415e-01])

In [ ]:
list(params.values())

[97.325616, 3.2143883, 0.98924241, 0.058514733, 0.2224634, 0.53702415]

## Maximum posterior estimator

In [ ]:
Q_MAP = -2 * loglike_BAO(list(params.values())) - np.shape(cov_data_BAO)[0] * np.log(2*np.pi) - np.log(np.linalg.det(cov_data_BAO))
Q_MAP

23.408073320874223

{'H0': 97.325616,
 'logA': 3.2143883,
 'ns': 0.98924241,
 'ombh2': 0.058514733,
 'omch2': 0.2224634,
 'tau': 0.53702415}